In [7]:
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import pandas as pd
import plotly.express as px
import quandl
import requests
from io import StringIO
from jupyter_dash import JupyterDash

In [8]:
app = JupyterDash(title="Víctor Luque - Quandl Dash App")
quandl.ApiConfig.api_key = ""

In [9]:
wiki_companies_text = requests.get("https://static.quandl.com/coverage/WIKI_PRICES.csv").text
wiki_companies_df = pd.read_csv(StringIO(wiki_companies_text))
wiki_companies_df.rename(columns={'ticker': 'Ticker'}, inplace=True)
wiki_companies_df.head()

,Ticker
0,A
1,AA
2,AAL
3,AAMC
4,AAN


In [10]:
data_secwiki = requests.get("https://raw.githubusercontent.com/BangaloreSharks/SharkStock/master/secwiki_tickers.csv").text
df_secwiki = pd.read_csv(StringIO(data_secwiki))
df_secwiki = df_secwiki[df_secwiki["Price"].notna()]
df_secwiki.head()

,Ticker,Name,Sector,Industry,Price,Collection
0,A,Agilent Technologies Inc.,Healthcare,Medical Laboratories & Research,WIKI/A,stocks/a
1,AA,Alcoa Inc.,Basic Materials,Aluminum,WIKI/AA,stocks/aa
3,AAN,"Aaron's, Inc.",Services,Rental & Leasing Services,WIKI/AAN,stocks/aan
4,AAON,AAON Inc.,Industrial Goods,General Building Materials,WIKI/AAON,stocks/aaon
5,AAP,Advance Auto Parts Inc.,Services,Auto Parts Stores,WIKI/AAP,stocks/aap


In [11]:
df_valid = pd.merge(wiki_companies_df, df_secwiki, on="Ticker")
#Remove Industry, Price and Collection columns
df_valid = df_valid.drop(columns=["Industry", "Price", "Collection"])
df_valid.head()

,Ticker,Name,Sector
0,A,Agilent Technologies Inc.,Healthcare
1,AA,Alcoa Inc.,Basic Materials
2,AAN,"Aaron's, Inc.",Services
3,AAON,AAON Inc.,Industrial Goods
4,AAP,Advance Auto Parts Inc.,Services


In [12]:
# Get data from Quandl
def get_quandl_data(ticker, start_date, end_date):
    response = quandl.get(f"WIKI/{ticker}.4", start_date=start_date, end_date=end_date)
    # Add a column with the ticker
    response["Ticker"] = ticker
    return response

In [13]:
@app.callback(
    Output(component_id='output', component_property='figure'),
    [Input(component_id='input_ticker', component_property='value'),
     Input(component_id='date_range', component_property='start_date'),
     Input(component_id='date_range', component_property='end_date')]
)
def create_figure(input_ticker:str, start_date:str, end_date:str):
    if input_ticker is not None:
        input_ticker = input_ticker.upper()
    else:
        input_ticker = 'GOOGL'
    # Collect data from Quandl
    # Create a figure
    df_quandl = get_quandl_data(input_ticker, start_date, end_date)
    # Move date index to a column
    df_quandl.reset_index(inplace=True)
    df_quandl.rename(columns={'Date': 'Date_Quandl'}, inplace=True)
    # Include rows from df_valid where ticker is the same as ticker from Quandl
    df_quandl = pd.merge(df_valid, df_quandl, on="Ticker")
    figure = px.line(
        df_quandl, x="Date_Quandl", y="Close",
        title="Precios de {}".format(df_valid[df_valid["Ticker"] == input_ticker]["Name"].to_string(index=False)),
        labels={
            "Close": "Precio de cierre (USD)",
            "Date_Quandl": "Fecha"
        },
        template="plotly_dark"
    )
    return figure

In [14]:
#create temp_df with tickers and names deleting notna
temp_df = df_valid[df_valid["Ticker"].notna()]
temp_df = df_valid[df_valid["Name"].notna()]
# Create a list of tickers
lista = temp_df[["Ticker", "Name"]].values
opciones = []
for item in lista:
    # Remove special characters from each item list of strings
    opciones.append({"label": item[1], "value": item[0]})

In [15]:
app.layout = html.Div(children=[
    html.H1(children='Víctor Luque - Dash App'),
    html.H3(children='''
        Una aplicación de Dash para mostrar la información del sépalo de las flores iris contenidas en Iris Dataset.
    '''),
    html.P("Seleccione una especie para ver su sépalo (setosa, version, virginica, todas)"),
    dcc.Dropdown(
        id='input_ticker',
        options=opciones,
        clearable=False,
        value='GOOG'
    ),
    dcc.DatePickerRange(
        id='date_range',
        min_date_allowed="2015-01-01",
        max_date_allowed="2021-12-31",
        start_date="2015-01-01",
        end_date="2021-12-31"

    ),
    # Create a figure
    dcc.Graph(id='output', figure=create_figure(input_ticker='GOOG', start_date='2015-01-01', end_date='2021-12-31')),
])

In [16]:
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [17]:
app.run_server(debug=True, mode="inline")